In [3]:
from transformers import AutoTokenizer
import re
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper, ContinuousColorMapper, LinearColorMapper, ColorBar, LabelSet, Label
from bokeh.plotting import figure, show, output_notebook
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.cluster import AgglomerativeClustering
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage
from gensim.models import KeyedVectors
from yellowbrick.cluster import KElbowVisualizer
from bs4 import BeautifulSoup
from sklearn.preprocessing import StandardScaler, normalize
import requests
import pandas as pd
from gensim.test.utils import common_texts
import gensim.downloader
import nltk
import seaborn as sns
from nltk.corpus import stopwords
from matplotlib import pyplot as plt
from bokeh.palettes import Spectral10
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import string
import sys
from nltk.tokenize import word_tokenize, sent_tokenize
import ipywidgets as widgets
import umap
from sklearn.feature_extraction.text import CountVectorizer

- gegeven de distributie van woorden, wat als je alle worden weg filter welke in het middelpunt van de curve vallen (66%)

gegeven een lijst met keywords, hoe kunnen we topics uit de documenten halen welke van toepassing zijn op de keywords?

# Research question 2

> *"Welke KPI's zitten verweven in de aangeleverde overheid documenten (denk aan geplande aantal huizen bijvoorbeeld), en wat zijn de overeenkomsten tussen deze KPI's? "*

In [45]:
import spacy
import warnings
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import wordpunct_tokenize
from tqdm import tqdm
from difflib import SequenceMatcher

lemmatizer = WordNetLemmatizer()
spacy_model = spacy.load('en_core_web_lg')

def preprocess_phrase(phrase):
    split_phrase = wordpunct_tokenize(phrase.lower())
    return " ".join([ lemmatizer.lemmatize(i) for i in split_phrase ])

def semantic_similarity(X, Y, max_out=None):
    # This function loads the large English language model in spaCy
    # and uses it to calculate semantic similarity between each phrase in Y and the input phrase X.
    # Note that this function assumes that the phrases in Y are in English, 
    # and it may not work as well for other languages or for very short or ungrammatical phrases.
    
    # Create a Doc object for X
    doc1 = spacy_model(preprocess_phrase(X))

    # Calculate semantic similarity for each phrase in Y
    similarities = []
    for phrase in Y:
        doc2 = spacy_model(preprocess_phrase(phrase))
        similarity = doc1.similarity(doc2)
        similarities.append((doc2, similarity))
        
    # return similarities
    if not max_out:
        max_out = len(similarities)
    return sorted(similarities, key=lambda a: a[1], reverse=True)[0:max_out]

def syntactic_similarity(X, Y, max_out=None):
    # This function takes in two parameters: 
    # X, the single phrase to compare against the list of phrases
    # Y, the list of phrases to compare against. 
    # It returns a tuple containing the most similar phrase from Y and the corresponding syntactic similarity score.
    match_ratios = []
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for phrase in Y:
            matcher = SequenceMatcher(None, preprocess_phrase(X), preprocess_phrase(phrase))
            ratio = matcher.ratio()
            match_ratios.append((phrase, ratio))
    
    if not max_out:
        max_out = len(match_ratios)
        
    return sorted(match_ratios, key=lambda a: a[1], reverse=True)[0:max_out]

In [3]:
handmatige_kpi_collectie = pd.read_csv("handmatige_kpi_collectie.csv")
handmatige_kpi_collectie.head(3)

,KPI,Document,Bladzijde
0,In het centrum bouwen we ruim 10.000 woningen ...,Omgevingsvisie Breda 2040 Sterk en Veerkrachti...,9
1,We verbeteren de verbindingen voor voetgangers...,Omgevingsvisie Breda 2040 Sterk en Veerkrachti...,9
2,De noordelijke rondweg ondertunnelen we groten...,Omgevingsvisie Breda 2040 Sterk en Veerkrachti...,9


In [4]:
handmatige_kpi_collectie.shape

(90, 3)

## KPI similaritie measarumnets

In [5]:
kpis = list(handmatige_kpi_collectie.KPI.astype(str))
synt_syms = dict()
for kpi in tqdm(kpis):
    others = [ i for i in kpis if i != kpi ]
    synt_syms[kpi] = syntactic_similarity(kpi, others, 5)
    
with open("./synt_sema.txt", "w", encoding="utf-8") as file:
    content = "<===> KPI + 5 meest overheenkomende KPI's (syntactisch) <===>\n\n"
    for i in synt_syms:
        content += i + "\n"
        for sym in synt_syms[i]:
            content += f"- {round(sym[1], 2)}\t{sym[0]}\n"
        content += "\n"
        
    file.write(content)

100%|██████████████████████████████████████████████████████████████████████████████████| 90/90 [00:07<00:00, 11.49it/s]


In [6]:
sema_syms = dict()
for kpi in tqdm(kpis):
    others = [ i for i in kpis if i != kpi ]
    sema_syms[kpi] = semantic_similarity(kpi, others, 5)
    
with open("./sema_syms.txt", "w", encoding="utf-8") as file:
    content = "<===> KPI + 5 meest overheenkomende KPI's (semantisch) <===>\n\n"
    for i in sema_syms:
        content += i + "\n"
        for sym in sema_syms[i]:
            content += f"- {round(sym[1], 2)}\t{sym[0]}\n"
        content += "\n"
        
    file.write(content)

100%|██████████████████████████████████████████████████████████████████████████████████| 90/90 [00:40<00:00,  2.25it/s]


In [7]:
# rows = []
# html_out = widgets.HBox()

# for i in zip(synt_syms, sema_syms):
#     synt = i[0]
#     sema = i[0]
    
#     grid_layout = widgets.Layout(
#         grid_auto_columns="min-content"
#     )
#     container_layout = widgets.Layout(
#         width='auto',
#         border="2px solid red",
#         flex_flow='col auto'
#     )
    
#     synt_syms_title = widgets.HTML(f"<h3>{synt}</h3>")
#     synt_syms_score = widgets.VBox([widgets.Label("Score:")] + [widgets.Label(str(round(i[1], 2)), layout=label_layout) for i in synt_syms[synt]])
#     synt_syms_phrase = widgets.VBox([widgets.Label("KPI:")] + [widgets.Label(str(i[0]), layout=label_layout) for i in synt_syms[synt]])
#     synt_syms_container = widgets.VBox([
#         synt_syms_title,
#         widgets.HBox([synt_syms_score, synt_syms_phrase], layout=grid_layout)
#     ], layout=container_layout)
        
#     sema_syms_title = widgets.HTML(f"<h3>{sema}</h3>")
#     sema_syms_score = widgets.VBox([widgets.Label("Score:")] + [widgets.Label(str(round(i[1], 2))) for i in sema_syms[sema]])
#     sema_syms_phrase = widgets.VBox([widgets.Label("KPI:")] + [widgets.Label(str(i[0])) for i in sema_syms[sema]])
#     sema_syms_container = widgets.VBox([
#         sema_syms_title,
#         widgets.HBox([sema_syms_score, sema_syms_phrase], layout=grid_layout)
#     ], layout=container_layout)
    
        
#     html_out.children += (widgets.HBox([synt_syms_container, sema_syms_container]), )
#     break

# html_out

We hebben nu een lijst met 90 KPI's. Het idee is om ruis KPI's te verwijderen. Wanneer dit gedaan is maken we een functie die alle symantisch/syntactisch overheen komende zinnen uit overheids documenten extrapoleerd welke 90% of meer overheen komen.

### Ruis KPI bepaling
**Wat maakt een goede KPI om in deze functies te gebruiken?**

Een KPI moet het volgende bevatten:
- ...

Gegeven een KPI, welke chunck komt daar het meest mee overheen

---

In [6]:
import io
import requests
import PyPDF2

def split_pdf_pages(pdf_data):
    pages = []
    reader = PyPDF2.PdfFileReader(io.BytesIO(pdf_data))

    for page_number in range(reader.getNumPages()):
        pages.append(reader.getPage(page_number))

    return pages

In [9]:
urls = ["https://file.notion.so/f/s/49da8fb5-000b-47bf-9788-1dcbdfdf80f6/Omgevingsvisie_Breda_2040_Sterk_en_Veerkrachtig_-_01-07-2021.pdf?id=3be70ed6-2a1a-460a-b326-f0031a4c4d81&table=block&spaceId=ca274338-dfb2-4f65-823d-2acde6195fea&expirationTimestamp=1684312025948&signature=7j13LI3pQfJCwASUOlCiPF8JzLPy6zZmMUcQjkV3xIA&downloadName=Omgevingsvisie+Breda+2040+Sterk+en+Veerkrachtig+-+01-07-2021.pdf"]

In [15]:
kpis = list(handmatige_kpi_collectie.KPI.astype(str))

page_content = []
with open("./Definitief concept Omgevingsvisie Weert (20-08-2021).pdf", "rb") as file:
    pdf_reader = PyPDF2.PdfFileReader(file)
    pages = []
    for page_number in tqdm(range(pdf_reader.getNumPages())):
        page = pdf_reader.getPage(page_number)
        page_content.append(page.extract_text())

100%|██████████████████████████████████████████████████████████████████████████████████| 41/41 [00:00<00:00, 42.34it/s]


In [16]:
phrases = re.split("\.|\n", "\n".join(page_content))
chuncks = []
N_chunks = 3
while len(phrases) >= N_chunks:
    chunk = " ".join(phrases[0:N_chunks])
    chuncks.append(chunk)
    phrases = phrases[1:]

In [17]:
chunk_synt_syms = dict()
for kpi in tqdm(kpis):
    chunk_synt_syms[kpi] = syntactic_similarity(kpi, chuncks, 5)

100%|██████████████████████████████████████████████████████████████████████████████████| 90/90 [02:45<00:00,  1.84s/it]


In [21]:
for i in chunk_synt_syms:
    print(i)
    for sim_chunck in chunk_synt_syms[i]:
        print(sim_chunck)
    print("----------------------------")

In het centrum bouwen we ruim 10.000 woningen en zorgen we ervoor dat hier alles goed bereikbaar is.
('gemiddelde woningdichtheid in het bebouwde gebied van Weert is 15 woningen per ha  Dit  maakt Weert als geheel suburbaan', 0.4636363636363636)
('  Een centrumstedelijk woonmilieu (> 70 woningen per ha) komt in Weert niet voor  De ', 0.4540540540540541)
('  Over het algemeen zijn deze woningen slecht geïsoleerd  Hier ligt een enorme ', 0.449438202247191)
('Over het algemeen zijn deze woningen slecht geïsoleerd  Hier ligt een enorme  verduurzamingsopgave', 0.44221105527638194)
('  Tot 2040 zullen daar zo’n 62 000 woningen moeten worden gebouwd', 0.42857142857142855)
----------------------------
We verbeteren de verbindingen voor voetgangers, over het water en via het spoor. We leggen mobiliteitshubs en een Singelpark aan.
('Belgische overheid bestaat de wens om deze verbinding geschikt te maken voor  personenvervoer  Niet alleen om Weert en Hamont met elkaar te verbinden, maar ook om ',

In [19]:
chunk_sema_syms = dict()
for kpi in tqdm(kpis):
    chunk_sema_syms[kpi] = semantic_similarity(kpi, chuncks, 5)

  0%|                                                                                           | 0/90 [00:00<?, ?it/s]C:\Users\luc\AppData\Local\Temp\ipykernel_17496\2445508710.py:28: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = doc1.similarity(doc2)
100%|██████████████████████████████████████████████████████████████████████████████████| 90/90 [15:57<00:00, 10.63s/it]


In [20]:
for i in chunk_sema_syms:
    print(i)
    for sim_chunck in chunk_sema_syms[i]:
        print(sim_chunck)
    print("----------------------------")

In het centrum bouwen we ruim 10.000 woningen en zorgen we ervoor dat hier alles goed bereikbaar is.
(2 weert : hoe staan we ervoor ? om een goed beeld van de weerter leefomgeving te krijgen , gaan we wat dieper in op de weerter waarden, 0.9024362837909564)
(die uitdagingen is om te komen tot een energieneutraal mobiliteitssysteem de verwachting is dat we in de toekomst een mobiliteitssysteem kennen waarin nog maar weinig mensen, 0.9004954561153791)
(3 2 weert : hoe staan we ervoor ? om een goed beeld van de weerter leefomgeving te krijgen , gaan we wat dieper in op de, 0.8992165435178068)
(en hoewel het aantal voertuigen op de weg in absolute aantallen nog niet eens zo sterk is afgenomen , zien we dat de file in deze periode zo goed al verdwenen waren, 0.8990567637955778)
(dan uitzondering en hoewel het aantal voertuigen op de weg in absolute aantallen nog niet eens zo sterk is afgenomen , zien we dat de file in deze periode zo goed al verdwenen, 0.8956213429364753)
------------------

Na aanleiding van bovenstaande uitkomsten kunnen wij concluderen dat het gebruik van semantische overheenkomst meting beter werkt dan syntactische metingen.

Er moet wel gekeken worden naar:
- andere manier van chunck verwerking (groter maken, minder overlapping etc)
- kwalitiet van sommige KPI's is soms twijfelbaar, er moet nader gekeken worden welke handmatig gekozen KPI's nouw echt het beste resultaat geven

---

In [9]:
from tqdm import tqdm


# get file content
page_content = []
with open("./Definitief concept Omgevingsvisie Weert (20-08-2021).pdf", "rb") as file:
    pdf_reader = PyPDF2.PdfFileReader(file)
    pages = []
    print("Getting page content")
    for page_number in tqdm(range(pdf_reader.getNumPages())):
        page = pdf_reader.getPage(page_number)
        page_content.append(page.extract_text())
        
        
# read KPI collection
kpis = list(pd.read_csv("handmatige_kpi_collectie.csv").KPI.astype(str))

Getting page content


100%|██████████████████████████████████████████████████████████████████████████████████| 41/41 [00:01<00:00, 34.70it/s]


To expend the list of KPI's, we measure the most syntactical similair chuncks of text from a given document. Because we don't know up front how many sentences the KPI extends to, we need to take chuncks of text from the document. These chuncks will be groups of N sentences. We have to allow for overlapping chuncks so that we are sure to measure entire KPI's. Then we measure the similairity between the handpicked KPI's and the given chuncks.

In [78]:
joined_content = re.sub("[\s\.\n]+", " ", "\n".join(page_content))
phrases = [ p for p in re.split("[\?\!\.\n]+", joined_content) if p ]
chuncks = []
N_chunks = 3

while len(phrases) >= N_chunks:
    chunk = " ".join(phrases[0:N_chunks])
    chuncks.append(chunk)
    phrases = phrases[1:]

Getting page content


'1Definitief concept Omgevingsvisie Weert (20-08-2021) Fase 0: Verzamelen en verkennen (Dit document wordt in de definitieve versie voorzien van een mooie opmaak met plaatjes, topo tijdreis en kaartjes) 2Inhoud 1 Waarom een Omgevingsvisie  4 1 1 Omgevingswet en Omgevingsvisie 4 1 2 Het proces 5 2 Werken aan Weert 2030 6 2 1 Weerter waarden 6 2 2 Sturen op effecten 6 2 3 Trends en ontwikkelingen 7 Grote opgaven in een beperkte ruimte 7 Bereikbaarheid 7 Klimaatopgaven 8 Economie 8 Gezondheid 10 Groen en landschap 10 Veiligheid 11 Maatschappelijke voorzieningen 12 Wonen 12 2 4 Profilering gemeente Weert in 2030 15 3 Weert in 2021 16 3 1 Weert in vogelvlucht 16 3 2 Weert: hoe staan we ervoor  17 Goed wonen voor iedere doelgroep 17 Iedereen doet mee 18 Natuurlijke en landschappelijke kwaliteit 18 Duurzaam en innovatief leven en ondernemen 19 Goed ontsloten en verweven met de regio 20 3 3 Regionale positionering 22 Rijk – NOVI 23 Provincie – POVI 24 Samenwerking Midden-Limburg (SML): “Midden

In [79]:
# store the chuncks which are sementicly similair to a given KPI
chunk_sema_syms = dict()

for kpi in tqdm(kpis):
    sema_chuncks = semantic_similarity(kpi, chuncks)
    
    # only store if the chunk and the KPI are 90% similair or more
    treshhold = 0.9
    chunk_sema_syms[kpi] = [ i for i in sema_chuncks if i[1] >= treshhold ]

100%|██████████████████████████████████████████████████████████████████████████████████| 90/90 [04:26<00:00,  2.96s/it]


In [103]:
parsed_chuncks = [ str(j[0]) for i in list(chunk_sema_syms.values()) for j in i if i ]

In [105]:
for i in parsed_chuncks:
    print(type(i))
    break

<class 'str'>


In [122]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.pipeline import Pipeline


vec = TfidfVectorizer(tokenizer=word_tokenize)
lda = LatentDirichletAllocation(n_components=5, random_state=0)

pipe = Pipeline([
    ("vec", vec),
    ("model", lda)
])

topic_matrix = pipe.fit_transform(parsed_chuncks)

D:\Users\luc\anaconda3\envs\sem4_base\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [113]:
lda_df = pd.DataFrame(topic_matrix, columns = pipe.get_feature_names_out())
lda_df.shape

(451, 5)